# Section 7: Harmonic Analysis using CAR maps

Now that users are able to import and examine data we present an introduction to harmonic analysis using the DR4 maps.  We will present an example of a simple ACT-Planck cross correlation but we encourage users to consider cross correlations with other maps or galaxy density maps.

To return to the main notebook or to view a different section use the links in the table of contents.

## Table of contents

 --- 
> [Section 1](Section_1_Introduction.ipynb): Introduction to AdvACT Data Release 4

> [Section 2](Section_2_coadded_maps.ipynb): S08-18 Coadded maps and an Introduction to Pixell

> [Section 3](Section_3_HEALPix_CAR_Conversions.ipynb):  Demonstration of methods to convert maps between HEALPix format and the CAR format.

> [Section 4](Section_4_visualize_objects.ipynb):  Guide to using Pixell to look at objects in the maps and stacking our maps on catalogue positions.

> [Section 5](Section_5_Lensing_maps.ipynb):  Introduction to the Lensing maps.

> [Section 6](Section_6_Component_separated_maps.ipynb):  Guide to the use of the Component Separated maps.

> [Section 7](Section_7_harmonic_analysis.ipynb):  Harmonic Analysis with CAR Maps

> [Section 8](Section_8_individual_patches_and_power_spectrum.ipynb): Maps and Power Spectra of Individual Patches 

> [Section 9](Section_9_E_B_maps.ipynb):  Look at how to generate E and B maps from I, Q, U maps. 
 
---

## Initialize notebook

For this notebook we need tools from the first notebook as well as from the [coadded_maps notebook](Section_2_coadded_maps.ipynb) which reads in the ACT map, and then the [HEALPix_CAR_Conversions notebook](HEALPix_CAR_Conversions.ipynb) which reads in a planck map and converts it from HEALPix format to the CAR format used for ACT maps.

Since the 3rd notebook runs the other two we just run the 3rd one here

These commands output the maps produced in the other notebooks so if you would like to see a more in-depth explanation of how these maps are formed feel free to navigate to those notebooks by using the links above.


In [ ]:
%run Section_3_HEALPix_CAR_Conversions.ipynb

## Begin Harmonic Analysis

### Fourier transforms using Enmap

We will begin by trimming both ACT and Planck maps to the same size, and then we use enmap's functions to Fourier transform the map using the 'flat-sky' approximation.  

Before Fourier transforming the maps it is necessary to apodize the edges of the maps so that periodic boundary conditions apply.  This can easily be done using the enmap library of functions. The taper can be adjusted by changing the 'apod_pix' variable which adjusts the number of pixels that are apodized.

Here we chose to display only the I maps, however, the Q and U maps can of course be tapered in the same manner if desired.

In [ ]:
# Trim Maps
box = np.array([[-28,40],[-12,-40]])*utils.degree
smap_act    = smap_2.submap(box)
smap_planck = planck_map.submap(box)

# Define taper using enmap.apod and then plot the taper
apod_pix = 100
taper    = enmap.apod(smap_act*0+1,apod_pix) #Ensures the taper map has the same shape and wcs as the data
print("Apodization")
eshow(taper,  **{"downgrade": 4, "ticks": 2})

# Apply taper to the two maps by multiplying the taper and maps together
print("Apodized ACT map")
eshow(taper*smap_act, **{"downgrade": 4,  "ticks": 2})

print("Apodized Plank map")
eshow(taper*smap_planck[0], **{"downgrade": 4,  "ticks": 2})

Now that we have the apodized maps we can begin the harmonic analysis.  

Step one is to use enmap to get a Fourier transformation of the map.  For ease of use we set the normalization of the Fourier transformation to physical by passing the 'phys' parameter.  This makes Pixell take into account pixel area factors when calculating the power, and results in power spectra with steradians as the unit of area.

Once you have the Fourier transformed map it is possible to make a naive power spectra by multiplying by the transform conjugate and then binning the spectra.

One thing we note is that Pixell has a function that lets you call the 'modlmap' of a ndmap.  This is a map that gives the distance in Fourier space of each pixel to the center of the map.  It's particularly useful for binning the power spectra and we will use it in the binning function below.

In order to get the Planck maps in $\mu$K we multiply by a factor of $10^6$ [this is fixed in Section 3 now in the latest version of pixell.reproject] 

In [ ]:
# Fourier transform the map:
kmap_act = enmap.fft(smap_act*taper, normalize = 'phys')
kmap_planck = enmap.fft(smap_planck[0]*taper, normalize = 'phys')

# Get cross power
cross_power = abs(kmap_act*np.conj(kmap_planck))

# Define a function that can bin the power by Fourier component magnitude
# This is the flat sky equivalent of "ell"
def bin(data, imap, lmax, bin_size):
    modlmap = imap.modlmap()

    # Define the bins and bin centers
    bins = np.arange(0, lmax, bin_size)
    centers = (bins[1:] + bins[:-1])/2.

    # Bin the power spectrum 
    digitized = np.digitize(np.ndarray.flatten(modlmap), bins, right=True)
    binned = np.bincount(digitized, data.reshape(-1))[1:-1]/np.bincount(digitized)[1:-1]
    return centers, binned

# The Taper we applied earlier effects the power spectrum. 
# Pixell doesn't have the capability to fully account for the taper
# But we can get an approximate correction by dividing out the following term
w2 = np.mean(taper**2)

ell_b, binned_cross_power = bin(cross_power/w2, smap_act, lmax = 6000, bin_size = 40)

Now that we've binned the power spectrum we made we can plot it quickly to have a look at it.

In [ ]:
# Read in a camb fiducial spectrum for comparison
from pixell import powspec
camb_theory = powspec.read_spectrum(path + "camb_theory.dat")
cltt = camb_theory[0,0,:3000]
ls = np.arange(cltt.size)

# Now plot the quick powerspectrum we just made
plt.figure(figsize = (8,6))
plt.rc('font', family='serif')

plt.plot(ls, cltt*ls**2., lw=3, color='k', label = "fiducial")
plt.plot(ell_b, ell_b**2*binned_cross_power, marker="o", ls="none",
         label = "150 GHz ACT + Planck Coadd x 143 GHz Planck")
plt.yscale('log')
plt.xlabel('$\\ell$', fontsize = 15)
plt.ylabel('$D_{\\ell}$', fontsize = 15)
plt.xlim(0, 5000)
plt.ylim(.1, 1e5)
plt.grid(True)
plt.legend(fontsize = 14)
plt.show()

We see two prominent features in this cross-spectrum: for intermediate $l$, there is some "missing" cross-power with respect to the fiducial curve. At high $l$, the data has a rising cross-power "tail," while the fiducial spectrum continues to fall off.

Generally speaking, the former feature is due to a combination of the damping effect of the various transfer functions in the experiments (e.g. Planck's wider beam), which we haven't accounted for. The latter captures the contribution from small-scale features in the maps, such as point sources and high-frequency noise. 

As a quick demonstration of the difference in the two experiments, we can plot the ACT-only power spectrum (i.e. the ACT autospectrum).

In [ ]:
# Get ACT power
auto_power = abs(kmap_act*np.conj(kmap_act))

ell_b, binned_auto_power = bin(auto_power/w2, smap_act, lmax = 6000, bin_size = 40)

# Plot the ACT autospectrum
plt.figure(figsize = (8,6))
plt.rc('font', family='serif')

plt.plot(ls, cltt*ls**2., lw=3, color='k', label = "fiducial")
plt.plot(ell_b, ell_b**2*binned_cross_power, marker="o", ls="none",
         label = "150 GHz ACT + Planck Coadd x 143 GHz Planck")
plt.plot(ell_b, ell_b**2*binned_auto_power, marker="o", ls="none",
         label = "150 GHz ACT + Planck Coadd Autospectrum")
plt.yscale('log')
plt.xlabel('$\\ell$', fontsize = 15)
plt.ylabel('$D_{\\ell}$', fontsize = 15)
plt.xlim(0, 5000)
plt.ylim(.1, 1e5)
plt.grid(True)
plt.legend(fontsize = 14)
plt.show()

As expected, we recover some of the intermediate $l$ power. On the other hand, we lose the benefit of having uncorrelated (really, less-correlated) noise between the two maps, so the power of the high $l$ tail has increased.

## Spherical harmonics using Pixell

We turn now to Pixell's curvedsky functions that enable spherical harmonic analysis of the patches.  The process is very similar to what one would do using the HEALPix library.  We will get alms from the maps; from there we get the cls from the alms. 

Again we want to weight our power spectrum by an area correction factor as well as the taper we've applied.

In [ ]:
from pixell import curvedsky 

# First we get the alms
alms_act = np.cdouble(curvedsky.map2alm(smap_act*taper, lmax = 6000))
alms_planck = np.cdouble(curvedsky.map2alm(smap_planck*taper, lmax = 6000)[0])

# Next step we get the cls using healpy
cls = hp.alm2cl(alms_act, alms_planck)

# We also want the corresponding ells which we can get easily
ell = np.arange(len(cls))

# Now we calculate the weighting we have to apply to our spectrum due to the 
# trimmed size
w2_new = enmap.area(smap_act.shape, smap_act.wcs)/4./np.pi

# Now we combine these to get the Dls
Dl = cls*ell**2 / w2_new 

In [ ]:
# Now we can plot our power spectrum
plt.figure(figsize = (8,6))
plt.rc('font', family='serif')
plt.grid(True)
plt.plot(ls,cltt*ls**2.,lw=3,color='k', label = "fiducial")
plt.plot(ell_b,ell_b**2*binned_power,marker="o",ls="none", label = "Fourier Transform")
plt.plot(ell, Dl, '.', label = "Spherical harmonics")
plt.yscale('log')
plt.xlabel('$\\ell$', fontsize = 15)
plt.ylabel('$D_{\\ell}$', fontsize = 15)
plt.title("Placnk 143 GHz x ACT + Planck 150 GHz Coadd", fontsize = 15)
plt.ylim(.01 ,1e6)
plt.legend(fontsize = 14)
plt.show()

In the plot above we can quite clearly see some of the acoustic oscillations as we would expect, and the agreement with the fiducial spectrum has improved versus the flat sky approximation at high $l$. However, there are still a few glaring problems.  Firstly, the spectrum  hasn't been corrected for beams or other pixel transfer functions both of which would need to be done for a true analysis.  This effect is easily noticeable at intermediate $l$ where our power spectra underestimates the fiducial spectrum. Also, we didn't mask point sources which can be done using the correct Planck mask and the source free ACT maps (the spectrum still plateaus at high $l$).

Furthermore we account for the window function using a relatively simple approach, however, to truly decouple it one should use a program such as **Namaster**, **pspy** or **pitas** which explicitly decouple the window functions.

Finally it's important to realize that calculating the error for any cross correlations would need to include knowledge of the noise model which is currently not complete.  

Despite the limitations, quick and useful power spectra can be made relatively easily using this approach and can be done with galaxy density maps or other CMB maps.

### Continue to the next section

[Section 8](Section_8_individual_patches_and_power_spectrum.ipynb): Maps and Power Spectra of Individual Patches 